In [37]:
!python /kaggle/working/sentiment_analysis/summarization/summarize_feedback.py


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-10-30 13:09:30.330046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761829770.352609   21371 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761829770.359864   21371 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
tokenizer_config.json: 100%|███████████████| 2.32k/2.32k [00:00<00:00, 11.8MB/s]
spiece.model: 100%|██████████████████████████| 792k/792k [00:00<00:00, 16.9MB/s]
tokenizer.json: 100%|██████████████████████| 1.39M/1.39M [00:00<00:00, 25.1MB/s]
config.json: 100%|█████████████████████████| 1.21k/1.21k [00:00<00:00, 8.09MB/

In [38]:
import pandas as pd
from IPython.display import display, HTML

SUM_DIR = "/kaggle/working/sentiment_analysis/summarization"
preview = pd.read_csv(f"{SUM_DIR}/summaries_preview.csv")
display(preview.head(10))


,id,source,clean_text,extractive_short,extractive_long,abstractive_short,abstractive_long
0,14080,twitter,trying call day get hung every time guy horrible,trying call day get hung every time guy horrible,trying call day get hung every time guy horrible,call day get hung every time guy horrible. try...,call day get hung every time guy horrible. try...
1,45939,yelp,place packed since place eat terminal one wait...,place packed since place eat terminal one wait...,place packed since place eat terminal one wait...,place packed since place eat terminal one wait...,place packed since place eat terminal one wait...
2,22075,yelp,absolutely love coming wrangler game n ngoing ...,absolutely love coming wrangler game n ngoing ...,absolutely love coming wrangler game n ngoing ...,wrangler announcer get audience involved thing...,wrangler announcer get audience involved thing...
3,16083,yelp,wish could negative star crook liar rape chanc...,wish could negative star crook liar rape chanc...,wish could negative star crook liar rape chanc...,wish could negative star crook liar rape chanc...,wish could negative star crook liar rape chanc...
4,24866,yelp,love hotel nice spa huge wave pool good sized ...,love hotel nice spa huge wave pool good sized ...,love hotel nice spa huge wave pool good sized ...,love hotel nice spa huge wave pool good sized ...,nice spa huge wave pool good sized lazy river ...
5,10543,twitter,american eagle plane using clt rdu disgusting ...,american eagle plane using clt rdu disgusting ...,american eagle plane using clt rdu disgusting ...,american eagle plane using clt rdu. disgusting...,american eagle plane using clt rdu. disgusting...
6,24986,yelp,lucky enough get table strolling course dresse...,lucky enough get table strolling course dresse...,lucky enough get table strolling course dresse...,lucky enough get table strolling course dresse...,lucky enough get table strolling course dresse...
7,38543,yelp,favorite bar go right mountain edge easy get a...,favorite bar go right mountain edge easy get a...,favorite bar go right mountain edge easy get a...,favorite bar go right mountain edge easy get a...,favorite bar go right mountain edge easy get a...
8,8863,twitter,delayed flight,delayed flight,delayed flight,delayed flight to london.,delayed flight from afghanistan to a syria.
9,1254,twitter,two year old birthdate,two year old birthdate,two year old birthdate,two year old birthdate.,"two year old birthdate. birthdate: 2 year old,..."


In [39]:
def show_examples(df, n=8):
    rows = df.sample(n=n, random_state=42).reset_index(drop=True)
    html = "<table border='1' style='width:100%'><tr><th>id</th><th>source</th><th>clean_text</th><th>extractive_short</th><th>abstractive_short</th><th>extractive_long</th><th>abstractive_long</th></tr>"
    for _, r in rows.iterrows():
        html += f"<tr><td>{r['id']}</td><td>{r['source']}</td><td>{r['clean_text'][:300]}</td><td>{r['extractive_short']}</td><td>{r['abstractive_short']}</td><td>{r['extractive_long']}</td><td>{r['abstractive_long']}</td></tr>"
    html += "</table>"
    display(HTML(html))

show_examples(preview, n=8)


id,source,clean_text,extractive_short,abstractive_short,extractive_long,abstractive_long
42416,yelp,patio great peolple watching beer cold see like food never crossed mind ever eat drink good time,patio great peolple watching beer cold see like food never crossed mind ever eat drink good time,patio great peolple watching beer cold see like food never crossed mind ever eat drink good time.,patio great peolple watching beer cold see like food never crossed mind ever eat drink good time,patio great peolple watching beer cold see like food never crossed mind ever eat drink good time.
9475,twitter,time bracket call fix problem attempting fix contact information issue day,time bracket call fix problem attempting fix contact information issue day,time bracket call fix problem attempting fix contact information issue day.,time bracket call fix problem attempting fix contact information issue day,time bracket call fix problem attempting fix contact information issue day. call fix issue.
15146,yelp,wife best labor day week end first car heat saturday night drop garage repair borrows car decides shred back tire sunday night raging thunderstorm one open fix holiday week end without vehicle nmy wife reserve car line website say major credit card rent car daughter pick u drive minute airport wait,wife best labor day week end first car heat saturday night drop garage repair borrows car decides shred back tire sunday night raging thunderstorm one open fix holiday week end without vehicle nmy wife reserve car line website say major credit card rent car daughter pick u drive minute airport wait line told present debit card need major credit card rent car wtf debit card cash turned car rental nmy wife informed clerk used debit card city rent car business trip point take condescending attitude told u renting car debit card ni kept cool clerk starting waving wife debit card face treating u like white trash credit card nat point wife lost profanely told thought enterprise care rental wife snatched debit card hand stormed mad hell nmy wife care major credit card pay cash something buy ni cannot believe today world company take debit card purchase nif enterprise last car rental place face earth car walk rather rent,wife best labor day week end first car heat saturday night drop garage repair borrows car decides shred back tire sunday night,wife best labor day week end first car heat saturday night drop garage repair borrows car decides shred back tire sunday night raging thunderstorm one open fix holiday week end without vehicle nmy wife reserve car line website say major credit card rent car daughter pick u drive minute airport wait line told present debit card need major credit card rent car wtf debit card cash turned car rental nmy wife informed clerk used debit card city rent car business trip point take condescending attitude told u renting car debit card ni kept cool clerk starting waving wife debit card face treating u like white trash credit card nat point wife lost profanely told thought enterprise care rental wife snatched debit card hand stormed mad hell nmy wife care major credit card pay cash something buy ni cannot believe today world company take debit card purchase nif enterprise last car rental place face earth car walk rather rent,wife best labor day week end first car heat saturday night drop garage repair borrows car decides shred back tire sunday night raging thunderstorm one open fix holiday week end without vehicle nmy wife reserve car line website say debit card need major credit card rent car wtf debit card cash turned car rental ni kept cool clerk starting waving wife debit
31205,yelp,family went chompies tempe today late lunch around pm large party people center room open booth near door hostess took u center noisy room asked seated booth away large party said enough wait staff cover table quieter section asked speak manager see could make exception waiting several minute hostes,family went chompies tempe today late lunch a

In [40]:
preview['len_text'] = preview['clean_text'].apply(lambda x: len(str(x).split()))
preview['len_ex_short'] = preview['extractive_short'].apply(lambda x: len(str(x).split()))
preview['len_abs_short'] = preview['abstractive_short'].apply(lambda x: len(str(x).split()))

print("Median original length:", preview['len_text'].median())
print("Median extractive short length:", preview['len_ex_short'].median())
print("Median abstractive short length:", preview['len_abs_short'].median())


Median original length: 28.5
Median extractive short length: 28.5
Median abstractive short length: 21.0


In [43]:
import pandas as pd

data = {
    "original_text": [
        "The product was great, delivery was fast, and customer support was friendly. I’ll definitely recommend it to my friends.",
        "Terrible experience! The app kept crashing and support was unhelpful. I want a refund."
    ],
    "reference_summary": [
        "Great product, fast delivery, and helpful support.",
        "App crashed often and support was poor."
    ],
    "generated_summary": [
        "Fast delivery and great support.",
        "Bad app experience with unhelpful support."
    ]
}

summary_df = pd.DataFrame(data)


In [44]:
from rouge_score import rouge_scorer
import numpy as np

gold_list = summary_df['reference_summary'].astype(str).tolist()
pred_list = summary_df['generated_summary'].astype(str).tolist()

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
scores = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(gold_list, pred_list)]

print("ROUGE-L fmeasure mean:", np.mean(scores))


ROUGE-L fmeasure mean: 0.4871794871794871


In [45]:
preview.to_csv(f"{SUM_DIR}/summaries_for_report.csv", index=False)
print("Saved summaries_for_report.csv")


Saved summaries_for_report.csv
